In [18]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hero777x_upi_fraud_dataset_path = kagglehub.dataset_download('hero777x/upi-fraud-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'upi-fraud-dataset' dataset.
Data source import complete.


In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/upi-fraud-dataset/UPI__DATA__SET.csv


In [20]:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df=pd.read_csv('/kaggle/input/upi-fraud-dataset/UPI__DATA__SET.csv')

In [22]:
df.head()

,Transaction_ID,Date,Time,Merchant_ID,Customer_ID,Device_ID,Transaction_Type,Payment_Gateway,Transaction_City,Transaction_State,IP_Address,Transaction_Status,Device_OS,Transaction_Frequency,Merchant_Category,Transaction_Channel,Transaction_Amount_Deviation,Days_Since_Last_Transaction,amount,fraud
0,T00022452,20-04-2023,04:50:22 AM,f65a902b-2396-40cc-9593-97e103f1bc15,89aaeceb-21f5-46c8-9de3-89dde6a10a75,c1e0deb4-7c97-4178-a838-38f4a2f0b57c,Refund,SamplePay,Durgapur,Chhattisgarh,140.213.7.48,Completed,MacOS,1,Brand Vouchers and OTT,In-store,25.02,5,396.62,1
1,T00032233,08-08-2023,08:09:21 AM,d4a5efcb-4eb6-4d3a-8132-07bb3e6e13a4,8a8962f9-e84a-4573-ab0b-187311978a21,62e14f64-b0ba-4284-889a-51ac05baf33e,Bank Transfer,SamplePay,Rajpur Sonarpur,Himachal Pradesh,184.108.177.45,Pending,Windows,30,Home delivery,Mobile,-36.64,20,121.94,1
2,T00037364,25-06-2023,07:49:01 AM,759ad138-9473-4729-8699-3d72c7ffb983,c3f53ce6-e305-4460-a71d-93bde26043ab,4281c542-ac77-4269-b4bb-1de93ac12677,Bank Transfer,Other,New Delhi,Himachal Pradesh,16.106.248.163,Failed,Android,2,Utilities,Online,44.19,22,106.69,1
3,T00015924,28-01-2024,05:44:12 AM,d8f561e4-bded-4ef0-bcd8-5494b2e31a94,9f0c5613-7d4c-4454-bee1-c47fbba406a1,058b6488-2469-42f2-bc7f-707921d35cad,Subscription,UPI Pay,Bharatpur,Chhattisgarh,65.245.160.212,Failed,Android,0,Purchases,Online,-54.34,28,3611.11,1
4,T00021805,21-06-2023,09:40:59 AM,26fdd7a1-8537-4dfe-bcf7-f5a127b36682,bf9aa6d5-7d3d-43f9-8631-31336720a383,068ff12c-127a-4cfb-9899-ddde37060b28,Investment,Dummy Bank,Sagar,Mizoram,33.172.152.38,Completed,MacOS,1,Other,Online,12.38,25,374.89,1


In [23]:
df.isnull().sum()

,0
Transaction_ID,0
Date,0
Time,0
Merchant_ID,0
Customer_ID,0
Device_ID,0
Transaction_Type,0
Payment_Gateway,0
Transaction_City,0
Transaction_State,0


In [24]:
df.dtypes

,0
Transaction_ID,object
Date,object
Time,object
Merchant_ID,object
Customer_ID,object
Device_ID,object
Transaction_Type,object
Payment_Gateway,object
Transaction_City,object
Transaction_State,object


In [25]:
num_cols=df.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols=df.select_dtypes(include=['object']).columns.tolist()

In [26]:
print()

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
num_processor=Pipeline(
    steps=[
        ('scaler',StandardScaler())
    ]
)

In [28]:
from sklearn.preprocessing import OneHotEncoder
cat_processor=Pipeline(
    steps=[
        ('onehot',OneHotEncoder())
    ]
)

In [29]:
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer(
    transformers=[
        ('cat',cat_processor,cat_cols),
        ('num',num_processor,[f for f in num_cols if f!='fraud'])
    ]
)

In [30]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
est=VotingClassifier([('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
                     ('mlp',MLPClassifier(hidden_layer_sizes=(100, 50),
                                            activation='relu',
                                            solver='adam',
                                            max_iter=300,
                                            random_state=42))])
model=Pipeline([('ct',ct),('est',est)])
model

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num'...
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None, ...)),
                                              ('mlp',
                                               MLPClassifier(hidden_layer_sizes=(100,
                                                                                 50),
                                                             max_iter=300,
                                                             random_state=42))]))])

In [31]:
X=df.drop('fraud',axis='columns')
y=df['fraud']
print(y)

0       1
1       1
2       1
3       1
4       1
       ..
6995    1
6996    0
6997    1
6998    1
6999    0
Name: fraud, Length: 7000, dtype: int64


In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [33]:
model.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:13:19] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num'...
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None, ...)),
                                              ('mlp',
                                               MLPClassifier(hidden_layer_sizes=(100,
                                                                                 50),
                                                             max_iter=300,
                                                             random_state=42))]))])

In [34]:
y_pred = model.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0
